Halyna Trush.
Contact Information
Phone: +380954200758
Email: frolova.galka@gmail.com
LinkedIn: https://www.linkedin.com/in/halyna-trush/

This notebook implements a complete pipeline for Sentinel-2 satellite image matching using both classical and deep-learning approaches.
It reads .jp2 images from Google Drive or a local folder, automatically detects keypoints, and compares image pairs captured in different seasons.
The workflow includes ORB + RANSAC (classical matching) and LoFTR (Kornia) (deep feature matching), computing metrics such as the number of matches, inlier ratio, reprojection error, and runtime.
The interactive interface allows users to select image pairs and rerun analysis without editing the code, ensuring full reproducibility and easy testing.

In [ ]:
# Install dependencies (run once in Colab)
!pip install -q rasterio kornia

import os
import cv2
import numpy as np
import torch
import kornia as K
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import reshape_as_image
!pip install rasterio
from google.colab import drive


In [ ]:
def read_image_any(path):
    """
    Read an image from a given path.

    - If extension is .jp2: read with rasterio (Sentinel-2 TCI, etc.).
    - Otherwise: read with OpenCV and convert BGR -> RGB.

    Returns:
        img (np.ndarray): RGB image of shape (H, W, 3), dtype uint8.
    """
    ext = os.path.splitext(path)[1].lower()

    if ext == ".jp2":
        # Read multi-band raster data in (C, H, W) format
        with rasterio.open(path, "r") as src:
            img = src.read()

        # Convert to (H, W, C)
        img = reshape_as_image(img)

        # Rescale to uint8 range [0, 255]
        img = img.astype(np.float32)
        max_val = img.max()
        if max_val > 0:
            img = img / max_val * 255.0
        img = img.clip(0, 255).astype(np.uint8)

    else:
        # Fallback for non-JP2 formats: use OpenCV
        bgr = cv2.imread(path, cv2.IMREAD_COLOR)
        if bgr is None:
            raise ValueError(f"Cannot read image: {path}")
        img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    return img


In [ ]:
def run_orb_matching(img1, img2, n_features=2000, ransac_thresh=5.0):
    """
    Classical pipeline:
    - ORB keypoints + descriptors
    - brute-force matching
    - RANSAC homography to keep geometrically consistent matches
    - performance metrics and visualization
    """
    import time

    start_time = time.time()

    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    # ORB detector
    orb = cv2.ORB_create(nfeatures=n_features)

    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    if des1 is None or des2 is None:
        print("No descriptors found in one of the images.")
        return

    # Brute-force matcher with Hamming distance
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda m: m.distance)

    raw_matches = len(matches)
    print(f"Raw ORB matches: {raw_matches}")

    if raw_matches < 4:
        print("Not enough matches for RANSAC.")
        return

    # Extract coordinates
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # RANSAC homography
    H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, ransacReprojThreshold=ransac_thresh)
    if H is None or mask is None:
        print("Homography could not be estimated.")
        return

    matches_mask = mask.ravel().tolist()
    inliers = int(np.sum(matches_mask))
    inlier_ratio = inliers / raw_matches if raw_matches > 0 else 0.0

    # Reprojection error for inliers
    pts1_inliers = pts1[matches_mask == np.array(1)]
    pts2_inliers = pts2[matches_mask == np.array(1)]
    if len(pts1_inliers) > 0:
        pts1_proj = cv2.perspectiveTransform(pts1_inliers, H)
        errors = np.linalg.norm(pts2_inliers - pts1_proj, axis=2)
        mean_error = float(errors.mean())
        median_error = float(np.median(errors))
    else:
        mean_error = np.nan
        median_error = np.nan

    end_time = time.time()
    runtime = end_time - start_time

    print(f"Inliers after RANSAC: {inliers} / {raw_matches} ({100 * inlier_ratio:.2f}%)")
    print(f"Mean reprojection error: {mean_error:.2f} px")
    print(f"Median reprojection error: {median_error:.2f} px")
    print(f"Runtime: {runtime:.2f} seconds")

    # Draw inlier matches only
    inlier_matches = [m for i, m in enumerate(matches) if matches_mask[i]]

    matched_img = cv2.drawMatches(
        img1, kp1,
        img2, kp2,
        inlier_matches,
        None,
        matchColor=(0, 255, 0),
        singlePointColor=None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    plt.figure(figsize=(14, 7))
    plt.imshow(matched_img)
    plt.title("ORB + RANSAC inlier matches")
    plt.axis("off")
    plt.show()


In [ ]:
class LoFTR_Matcher:
    def __init__(self, image_size=None, device=None):
        """
        image_size: optional (H, W) to resize images before matching.
        device: 'cuda' or 'cpu'; if None, choose automatically.
        """
        self.image_size = image_size
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.matcher = K.feature.LoFTR(pretrained="outdoor").eval().to(self.device)

    def _convert_image(self, image):
        """
        Convert H×W×3 uint8 RGB image to [1, 3, H, W] float tensor in [0, 1].
        Optionally resize if image is too large or image_size is specified.
        """
        SIZE_MAX = 1280
        img = K.utils.image_to_tensor(image)  # [3, H, W]
        img = img.float().unsqueeze(0).to(self.device) / 255.0

        if self.image_size is not None:
            img = K.geometry.resize(img, self.image_size, interpolation="area")
        elif max(img.shape[-2], img.shape[-1]) > SIZE_MAX:
            img = K.geometry.resize(img, SIZE_MAX, side="long", interpolation="area")

        return img

    def __call__(self, image0, image1, confidence_min=0.8):
        """
        Run LoFTR on two RGB uint8 images.

        Returns:
            dict with:
                keypoints0: (N, 2) array of points in image0
                keypoints1: (N, 2) array of points in image1
                confidence: (N,) array of match confidences
                inliers: (N, 1) boolean array (MAGSAC inliers)
                fundamental: 3x3 fundamental matrix or None
        """
        img0_t = self._convert_image(image0)
        img1_t = self._convert_image(image1)

        inp = {
            "image0": K.color.rgb_to_grayscale(img0_t),
            "image1": K.color.rgb_to_grayscale(img1_t),
        }

        with torch.inference_mode():
            corresp = self.matcher(inp)

        # Confidence filter
        mask = corresp["confidence"] > confidence_min
        idx = torch.nonzero(mask, as_tuple=True)

        keypoints0 = corresp["keypoints0"][idx].cpu().numpy()
        keypoints1 = corresp["keypoints1"][idx].cpu().numpy()
        confidence = corresp["confidence"][idx].cpu().numpy()

        fundamental = None
        try:
            fmat, inliers = cv2.findFundamentalMat(
                keypoints0,
                keypoints1,
                cv2.USAC_MAGSAC,
                1.0,
                0.99,
                100000
            )
            if inliers is None:
                inliers = np.zeros((keypoints0.shape[0], 1), dtype=bool)
            else:
                inliers = inliers.astype(bool)
            fundamental = fmat
        except Exception:
            inliers = np.zeros((keypoints0.shape[0], 1), dtype=bool)

        return {
            "keypoints0": keypoints0,
            "keypoints1": keypoints1,
            "confidence": confidence,
            "inliers": inliers,
            "fundamental": fundamental,
        }


In [ ]:
def run_loftr_matching(img1, img2, confidence_min=0.8, max_draw=200):
    """
    Run LoFTR on two images, print basic metrics and visualize inlier matches.

    Metrics:
        - total matched keypoints (after confidence filter)
        - number of inliers (MAGSAC)
        - inlier ratio
        - runtime (seconds)
    """
    import time

    matcher = LoFTR_Matcher()
    img0_u8 = img1.astype(np.uint8)
    img1_u8 = img2.astype(np.uint8)

    start_time = time.time()
    results = matcher(img0_u8, img1_u8, confidence_min=confidence_min)
    end_time = time.time()
    runtime = end_time - start_time

    num_total = len(results["keypoints0"])
    num_inliers = int(results["inliers"].sum()) if results["inliers"] is not None else 0
    ratio = num_inliers / num_total if num_total > 0 else 0.0

    print("\nLoFTR stats:")
    print(f"  Total matched keypoints: {num_total}")
    print(f"  Inliers (MAGSAC):       {num_inliers}")
    print(f"  Inlier ratio:           {ratio:.2f}")
    print(f"  Runtime:                {runtime:.2f} seconds")

    if num_inliers == 0:
        print("No inliers found, nothing to visualize.")
        return

    # Extract inlier coordinates
    inlier_mask = results["inliers"].ravel().astype(bool)
    kp0 = results["keypoints0"][inlier_mask]
    kp1 = results["keypoints1"][inlier_mask]

    # Random subsample for visualization
    if kp0.shape[0] > max_draw:
        sel_idx = np.random.choice(kp0.shape[0], max_draw, replace=False)
        kp0 = kp0[sel_idx]
        kp1 = kp1[sel_idx]

    # Build side-by-side canvas
    h0, w0, _ = img0_u8.shape
    h1, w1, _ = img1_u8.shape
    canvas_h = max(h0, h1)
    canvas_w = w0 + w1
    canvas = np.zeros((canvas_h, canvas_w, 3), dtype=np.uint8)
    canvas[:h0, :w0] = cv2.cvtColor(img0_u8, cv2.COLOR_RGB2BGR)
    canvas[:h1, w0:w0 + w1] = cv2.cvtColor(img1_u8, cv2.COLOR_RGB2BGR)

    # Draw matches in green
    for (x0, y0), (x1, y1) in zip(kp0, kp1):
        p1 = (int(x0), int(y0))
        p2 = (int(x1) + w0, int(y1))
        cv2.circle(canvas, p1, 3, (0, 255, 0), -1)
        cv2.circle(canvas, p2, 3, (0, 255, 0), -1)
        cv2.line(canvas, p1, p2, (0, 255, 0), 1)

    plt.figure(figsize=(18, 9))
    plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("LoFTR inlier matches (subset)")
    plt.show()


In [ ]:
import os

# If running in Google Colab, you can optionally mount Drive,
# but the notebook does not depend on it.
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Base directory for Sentinel-2 .jp2 files.
# In Colab: uses Google Drive (/content/drive/MyDrive/Sentinel) by default.
# For local testing: create ./data/Sentinel and place your .jp2 files there.
BASE_DIR = "/content/drive/MyDrive/Sentinel"

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Option 1: use data stored in the project directory on Colab
    # BASE_DIR = "/content/data/Sentinel"
    # Option 2: uncomment the line below if the user wants to use their own Drive folder:
    # BASE_DIR = "/content/drive/MyDrive/Sentinel"

print("Using data directory:", BASE_DIR)
if not os.path.isdir(BASE_DIR):
    print("Directory does not exist. Please create it and put Sentinel-2 .jp2 files there.")


In [ ]:
# Sentinel-2 image matching loop: read from Google Drive folder and process pairs

# Mount Google Drive
#drive.mount('/content/drive')

# Base directory with Sentinel-2 .jp2 files
#BASE_DIR = "/content/drive/MyDrive/Sentinel"

def list_jp2_files(base_dir):
    """Return sorted list of .jp2 files in the given directory."""
    files = [f for f in os.listdir(base_dir) if f.lower().endswith(".jp2")]
    files.sort()
    return files

while True:
    jp2_files = list_jp2_files(BASE_DIR)
    if len(jp2_files) < 2:
        print("Not enough .jp2 files in directory:", BASE_DIR)
        break

    print("\nAvailable .jp2 files:")
    for idx, name in enumerate(jp2_files):
        print(f"{idx}: {name}")

    # Select two files by index
    try:
        idx1 = int(input("\nEnter index of FIRST image: "))
        idx2 = int(input("Enter index of SECOND image: "))
    except ValueError:
        print("Invalid input. Please enter integer indices.")
        continue

    if not (0 <= idx1 < len(jp2_files)) or not (0 <= idx2 < len(jp2_files)):
        print("Index out of range. Try again.")
        continue
    if idx1 == idx2:
        print("Indices must be different. Try again.")
        continue

    img1_path = os.path.join(BASE_DIR, jp2_files[idx1])
    img2_path = os.path.join(BASE_DIR, jp2_files[idx2])

    print("\nSelected images:")
    print("Image 1:", jp2_files[idx1])
    print("Image 2:", jp2_files[idx2])

    # Read both images
    img1 = read_image_any(img1_path)
    img2 = read_image_any(img2_path)

    # Optional: downscale to reduce memory load
    scale_factor = 0.3
    if scale_factor != 1.0:
        img1_small = cv2.resize(img1, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)
        img2_small = cv2.resize(img2, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)
    else:
        img1_small, img2_small = img1, img2

    # Quick visualization
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(img1_small)
    plt.title(f"Image 1 ({jp2_files[idx1]})")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(img2_small)
    plt.title(f"Image 2 ({jp2_files[idx2]})")
    plt.axis("off")
    plt.show()

    # Run both algorithms
    print("\nRunning ORB + RANSAC...")
    run_orb_matching(img1_small, img2_small)

    print("\nRunning LoFTR matching...")
    run_loftr_matching(img1_small, img2_small)

    # Ask if user wants to continue
    ans = input("\nDo you want to process another pair of images? (y/n): ").strip().lower()
    if ans != "y":
        print("Stopping interactive loop.")
        break
